In [137]:
from math import *
from leg import *
from body import *
from control import *
from geometry import *
import numpy as np
from robot import init
from logger import Logger

In [138]:
Logger.init()
c = init()
b = c.body
c.set_posture('stand')
c.walk(10, 0)
print(b.position)
print('Done!')

INFO: Logger initialised at 2025-02-22 20:00:03.042556
INFO: Logger initialised at 2025-02-22 20:00:03.043420
INFO: adding gait 'default': fl,rr,fr,rl
INFO: goto leg 'fl' target(x=2.300 y=0.000 z=-2.000) angles cox 90.0 femur 0.0 tibia 90.0
INFO: goto leg 'fr' target(x=2.300 y=0.000 z=-2.000) angles cox 90.0 femur 0.0 tibia 90.0
INFO: goto leg 'rl' target(x=2.300 y=0.000 z=-2.000) angles cox 90.0 femur 0.0 tibia 90.0
INFO: goto leg 'rr' target(x=2.300 y=0.000 z=-2.000) angles cox 90.0 femur 0.0 tibia 90.0
INFO: Logger initialised at 2025-02-22 20:00:03.044640
INFO: goto leg 'fl' target(x=1.000 y=0.000 z=-2.500) angles cox 90.0 femur 34.6 tibia 77.2
INFO: goto leg 'fr' target(x=1.000 y=0.000 z=-2.500) angles cox 90.0 femur 34.6 tibia 77.2
INFO: goto leg 'rl' target(x=1.000 y=0.000 z=-2.500) angles cox 90.0 femur 34.6 tibia 77.2
INFO: goto leg 'rr' target(x=1.000 y=0.000 z=-2.500) angles cox 90.0 femur 34.6 tibia 77.2
INFO: control.walk distance 10
INFO: starting step at (x=0.000 y=0.000

In [139]:
t1 = Transform(zrot=20)
print(t1)
for ll in b.legs.values():
    print(ll.which, ll.location, ll.location@t1 - ll.location)

[[ 0.94  -0.342  0.     0.   ]
 [ 0.342  0.94   0.     0.   ]
 [ 0.     0.     1.     0.   ]
 [ 0.     0.     0.     1.   ]]
fl (x=85.000 y=50.000 z=0.000) (x=11.975 y=-32.087 z=0.000)
fr (x=85.000 y=-50.000 z=0.000) (x=-22.227 y=-26.056 z=0.000)
rl (x=-90.000 y=50.000 z=0.000) (x=22.529 y=27.766 z=0.000)
rr (x=-90.000 y=-50.000 z=0.000) (x=-11.673 y=33.797 z=0.000)


In [140]:
t2 = Transform(x=10)
for ll in b.legs.values():
    print(ll.which, ll.location, ll.location@(t2@t1) - ll.location)

fl (x=85.000 y=50.000 z=0.000) (x=21.372 y=-35.507 z=0.000)
fr (x=85.000 y=-50.000 z=0.000) (x=-12.830 y=-29.477 z=0.000)
rl (x=-90.000 y=50.000 z=0.000) (x=31.926 y=24.346 z=0.000)
rr (x=-90.000 y=-50.000 z=0.000) (x=-2.276 y=30.377 z=0.000)


In [141]:
t2 = Transform(x=10)
for ll in b.legs.values():
    print(ll.which, ll.location, ll.location@(t1@t2) - ll.location)

fl (x=85.000 y=50.000 z=0.000) (x=21.975 y=-32.087 z=0.000)
fr (x=85.000 y=-50.000 z=0.000) (x=-12.227 y=-26.056 z=0.000)
rl (x=-90.000 y=50.000 z=0.000) (x=32.529 y=27.766 z=0.000)
rr (x=-90.000 y=-50.000 z=0.000) (x=-1.673 y=33.797 z=0.000)


In [142]:
print(t1@t2)
print(t2@t1)
print('hello')

[[ 0.94  -0.342  0.     0.   ]
 [ 0.342  0.94   0.     0.   ]
 [ 0.     0.     1.     0.   ]
 [10.     0.     0.     1.   ]]
[[ 0.94  -0.342  0.     0.   ]
 [ 0.342  0.94   0.     0.   ]
 [ 0.     0.     1.     0.   ]
 [ 9.397 -3.42   0.     1.   ]]
hello


In [143]:
t3 = t2@t1
print(datan2(t3.y(), t3.x()))

-19.999999999999996


In [144]:
print(cosine_rule(1,1,sqrt(2)))
print(cosine_rule(1,sqrt(2),1))
print(cosine_rule(1,1,1))
print(cosine_rule(1,2,sqrt(3)))
print(cosine_rule(2,sqrt(3),1))

90.00000000000001
44.999999999999986
60.00000000000001
59.99999999999999
29.999999999999993


In [145]:
femur=2.0
tibia=2.0

def get_ft(toe_pos: Point2D) -> LegAngles:
        result = LegAngles()
        h = toe_pos.length()
        alpha = toe_pos.angle()
        beta = cosine_rule(h, femur, tibia)
        result.femur = 90 - (beta - alpha)
        result.tibia = cosine_rule(tibia, femur, h)
        #print(f'femur_tibia pos {toe_pos} {h=} {alpha=} {beta=} angles {result}')
        return result


In [146]:
p1=Point2D(1,sqrt(3))
print(p1)
print(get_ft(p1))

(x=1.000 y=1.732)
cox 0.0 femur 60.0 tibia 60.0


In [147]:
def get_angs(toe_pos: Point) -> LegAngles:
        cox = datan2(toe_pos.y(), -toe_pos.z())
        try:
            result = get_ft(Point2D(toe_pos.x(), -toe_pos.z() / dcos(cox)))
        except ValueError as exc:
            raise exc
        result.cox = cox + 90
        return result


In [148]:
print(get_angs(Point(1,1,-sqrt(3))))

cox 120.0 femur 60.6 tibia 68.0


In [149]:
    def get_tp2(angles: LegAngles) -> Point2D:
        knee_pos = Point2D(-femur * dcos(angles.femur), femur * dsin(angles.femur))
        #print(knee_pos)
        alpha = angles.tibia - angles.femur
        #print(angles, alpha)
        toe_offset = Point2D(tibia * dcos(alpha), tibia * dsin(alpha))
        #print(toe_offset)
        return (knee_pos + toe_offset).reflect_x()


In [150]:
x1=get_ft(p1)
print(get_tp2(x1))

(x=1.000 y=-1.732)


In [151]:
 def get_tp(angles: LegAngles) -> Point:
        t1 = get_tp2(angles)
        h = t1.y()
        return Point(t1.x(), -h * dsin(angles.cox - 90), h * dcos(angles.cox - 90)) 



In [152]:

a1=get_angs(Point(1,1,-sqrt(3)))
print()
print(a1)
print()
print(get_tp2(a1))
print()
print(get_tp(a1))


cox 120.0 femur 60.6 tibia 68.0

(x=1.000 y=-2.000)

(x=1.000 y=1.000 z=-1.732)


In [156]:
from itertools import product
x = [-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1]
y = x
z = [-1, -1.2, -1.4, -1.6, -1.8, -2, -2.2, -2.4, -2.6, -2.8]
for p in product(x,y,z):
    pp = Point(*p)
    a = get_angs(pp)
    ppp = get_tp(a)
    if True or ppp!= pp:
        print(pp, ppp, a)
print("Done!")

(x=-1.000 y=-1.000 z=-1.000) (x=-1.000 y=-1.000 z=-1.000) cox 45.0 femur -9.6 tibia 51.3
(x=-1.000 y=-1.000 z=-1.200) (x=-1.000 y=-1.000 z=-1.200) cox 50.2 femur -5.0 tibia 55.2
(x=-1.000 y=-1.000 z=-1.400) (x=-1.000 y=-1.000 z=-1.400) cox 54.5 femur -0.3 tibia 59.7
(x=-1.000 y=-1.000 z=-1.600) (x=-1.000 y=-1.000 z=-1.600) cox 58.0 femur 4.3 tibia 64.5
(x=-1.000 y=-1.000 z=-1.800) (x=-1.000 y=-1.000 z=-1.800) cox 60.9 femur 9.0 tibia 69.8
(x=-1.000 y=-1.000 z=-2.000) (x=-1.000 y=-1.000 z=-2.000) cox 63.4 femur 13.7 tibia 75.5
(x=-1.000 y=-1.000 z=-2.200) (x=-1.000 y=-1.000 z=-2.200) cox 65.6 femur 18.4 tibia 81.7
(x=-1.000 y=-1.000 z=-2.400) (x=-1.000 y=-1.000 z=-2.400) cox 67.4 femur 23.1 tibia 88.3
(x=-1.000 y=-1.000 z=-2.600) (x=-1.000 y=-1.000 z=-2.600) cox 69.0 femur 28.0 tibia 95.5
(x=-1.000 y=-1.000 z=-2.800) (x=-1.000 y=-1.000 z=-2.800) cox 70.3 femur 33.1 tibia 103.3
(x=-1.000 y=-0.800 z=-1.000) (x=-1.000 y=-0.800 z=-1.000) cox 51.3 femur -14.0 tibia 47.9
(x=-1.000 y=-0.800 z=